In [18]:
pip install focal-loss

In [28]:
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from datetime import datetime 
import cv2
from PIL import Image
from keras import backend, optimizers
from focal_loss import BinaryFocalLoss
import glob
import rasterio
import numpy as np
from keras.utils import normalize

In [50]:
train_mask="D:/Jintu/Jintu_LULC/TestDL_5/Data Preparation/Patchify/64_patches/lulc/"
train_ndvi="D:/Jintu/Jintu_LULC/TestDL_5/Data Preparation/Patchify/64_patches/ndvi/"
train_ndwi="D:/Jintu/Jintu_LULC/TestDL_5/Data Preparation/Patchify/64_patches/ndwi/"


In [51]:
#Capture training image info as a list
train_masks = []
# Assuming train_image is the directory pattern you want to search
for directory_path in glob.glob(train_mask):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        with rasterio.open(img_path) as src:
            img = src.read(1)  # Read the image as a numpy array (assuming it's a single-band image)
            train_masks.append(img)
#Convert list to array for machine learning processing        
train_masks = np.array(train_masks)

#Capture training image info as a list
train_ndvis = []
# Assuming train_image is the directory pattern you want to search
for directory_path in glob.glob(train_ndvi):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        with rasterio.open(img_path) as src:
            img = src.read(1)  # Read the image as a numpy array (assuming it's a single-band image)
            train_ndvis.append(img)
            
#Capture training image info as a list
train_ndwis = []
# Assuming train_image is the directory pattern you want to search
for directory_path in glob.glob(train_ndwi):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        with rasterio.open(img_path) as src:
            img = src.read(1)  # Read the image as a numpy array (assuming it's a single-band image)
            train_ndwis.append(img)
            
#Convert list to array for machine learning processing        
train_ndwis = np.array(train_ndwis)
print(train_ndwis)
            
#Convert list to array for machine learning processing        
train_ndvis = np.array(train_ndvis)
print(train_ndvis)
print(train_ndvis.shape)

train_ndvis= np.expand_dims(train_ndvis, axis=3)
train_ndvis = normalize(train_ndvis, axis=3)

train_ndwis= np.expand_dims(train_ndwis, axis=3)
train_ndwis = normalize(train_ndwis, axis=1)

[[[ 0.3291682   0.27968127  0.3567867  ...  0.32540697  0.35557264
    0.35557264]
  [ 0.23272586  0.2771192   0.2735982  ...  0.35730118  0.3415681
    0.33439717]
  [ 0.23069175  0.23069175  0.2917826  ...  0.34138244  0.33800027
    0.37672454]
  ...
  [ 0.56160396  0.54388714  0.5873753  ...  0.48962024  0.4733302
    0.4269691 ]
  [ 0.57008964  0.4806313   0.5158325  ...  0.46572718  0.43419653
    0.4806196 ]
  [ 0.54991084  0.47928995  0.4896868  ...  0.44864866  0.45850757
    0.47351563]]

 [[ 0.324511    0.37125894  0.36278826 ...  0.22416994  0.23662326
    0.2684797 ]
  [ 0.33100942  0.3402877   0.36278826 ...  0.2748493   0.22477587
    0.27924204]
  [ 0.2951677   0.34788033  0.35102805 ...  0.2733478   0.26654866
    0.24085073]
  ...
  [ 0.410156    0.4633826   0.44020358 ...  0.5152062   0.4372523
    0.4372523 ]
  [ 0.4513695   0.5215805   0.45291543 ...  0.41878876  0.46264857
    0.41713902]
  [ 0.4449294   0.4449294   0.48188654 ...  0.42289495  0.39005998
    0.423

In [52]:
#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 2, 3, 4, 5], dtype=int64)

In [53]:
print(train_masks_encoded_original_shape.shape)

(900, 64, 64)


In [54]:
train_masks = np.expand_dims(train_masks_encoded_original_shape, axis=3)

In [55]:
#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test1,X2,X_test2,y1, y_test = train_test_split(train_ndvis,train_ndwis, train_masks, test_size = 0.10, random_state = 0)

In [2]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2.0 * intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1.0)


def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [3]:
def conv_block(x, filter_size, size, dropout, batch_norm=False):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv

In [4]:
def repeat_elem(tensor, rep):
    # lambda function to repeat Repeats the elements of a tensor along an axis
    #by a factor of rep.
    # If tensor has shape (None, 256,256,3), lambda will return a tensor of shape 
    #(None, 256,256,6), if specified axis=3 and rep=2.

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)

In [5]:
def res_conv_block(x, filter_size, size, dropout, batch_norm=False):

    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation('relu')(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    #conv = layers.Activation('relu')(conv)    #Activation before addition with shortcut
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    shortcut = layers.Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)

    res_path = layers.add([shortcut, conv])
    res_path = layers.Activation('relu')(res_path)    #Activation after addition with shortcut (Original residual block)
    return res_path

In [6]:
def gating_signal(input, out_size, batch_norm=False):
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

In [7]:
def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('sigmoid')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn

In [8]:
def UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):

    # network structure
    FILTER_NUM = 64 # number of filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    

    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
   
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, conv_16], axis=3)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, conv_32], axis=3)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, conv_64], axis=3)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
   
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, conv_128], axis=3)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
   
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model 
    model = models.Model(inputs, conv_final, name="UNet")
    print(model.summary())
    return model

In [58]:
#13 inputs 

In [89]:
import tensorflow as tf
from tensorflow.keras import layers, models

def UNet(input_shape, num_inputs=2, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    # Network structure
    FILTER_NUM = 64  # Number of filters for the first layer
    FILTER_SIZE = 3  # Size of the convolutional filter
    UP_SAMP_SIZE = 2  # Size of upsampling filters

    input_layers = [layers.Input(input_shape, dtype=tf.float32) for _ in range(num_inputs)]

    # Process each input separately
    processed_inputs = []
    for i, input_layer in enumerate(input_layers):
        conv = conv_block(input_layer, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
        processed_inputs.append(conv)

    # Merge the processed inputs
    merged = layers.concatenate(processed_inputs, axis=3)

    # Upsampling layers
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE))(merged)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8 * FILTER_NUM, dropout_rate, batch_norm)
    
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE))(up_conv_16)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4 * FILTER_NUM, dropout_rate, batch_norm)
    
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE))(up_conv_32)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2 * FILTER_NUM, dropout_rate, batch_norm)
    
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE))(up_conv_64)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1, 1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  # Change to softmax for multichannel

    # Model
    model = models.Model(inputs=input_layers, outputs=conv_final, name="UNet")
    return model

def conv_block(input_layer, filter_size, num_filters, dropout_rate, batch_norm):
    # Define a convolutional block with optional dropout and batch normalization
    x = layers.Conv2D(num_filters, (filter_size, filter_size), activation='relu', padding='same')(input_layer)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    if dropout_rate > 0.0:
        x = layers.Dropout(dropout_rate)(x)
    x = layers.Conv2D(num_filters, (filter_size, filter_size), activation='relu', padding='same')(x)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    return x

# Example usage:
input_shape = (64, 64, 1)  # Adjust input shape as needed
num_inputs = 2  # Number of input tensors
model = UNet(input_shape, num_inputs=num_inputs, NUM_CLASSES=6, dropout_rate=0.0, batch_norm=True)
model.summary()


Model: "UNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_58 (InputLayer)       [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 input_59 (InputLayer)       [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_247 (Conv2D)         (None, 64, 64, 64)           640       ['input_58[0][0]']            
                                                                                                  
 conv2d_249 (Conv2D)         (None, 64, 64, 64)           640       ['input_59[0][0]']            
                                                                                               

                                                                                                  
 conv2d_258 (Conv2D)         (None, 1024, 1024, 64)       36928     ['batch_normalization_257[0][0
                                                                    ]']                           
                                                                                                  
 batch_normalization_258 (B  (None, 1024, 1024, 64)       256       ['conv2d_258[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 conv2d_259 (Conv2D)         (None, 1024, 1024, 6)        390       ['batch_normalization_258[0][0
                                                                    ]']                           
                                                                                                  
 batch_nor

In [92]:
def Attention_UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    
    inputs = layers.Input(input_shape, dtype=tf.float32)

    # Downsampling layers
    # DownRes 1, convolution + pooling
    conv_128 = conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=3)
    up_conv_16 = conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=3)
    up_conv_32 = conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=3)
    up_conv_64 = conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=3)
    up_conv_128 = conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=3)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="Attention_UNet")
    return model

# Example usage:
input_shape = (64, 64, 1)  # Adjust input shape as needed
num_inputs = 2  # Number of input tensors
model = Attention_UNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True)
model.summary()

Model: "Attention_UNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_60 (InputLayer)       [(None, 64, 64, 1)]          0         []                            
                                                                                                  
 conv2d_260 (Conv2D)         (None, 64, 64, 64)           640       ['input_60[0][0]']            
                                                                                                  
 batch_normalization_260 (B  (None, 64, 64, 64)           256       ['conv2d_260[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 conv2d_261 (Conv2D)         (None, 64, 64, 64)           36928     ['batch_normaliza

 conv2d_transpose (Conv2DTr  (None, 4, 4, 512)            2359808   ['conv2d_272[0][0]']          
 anspose)                                                                                         
                                                                                                  
 conv2d_271 (Conv2D)         (None, 4, 4, 512)            1049088   ['batch_normalization_267[0][0
                                                                    ]']                           
                                                                                                  
 add (Add)                   (None, 4, 4, 512)            0         ['conv2d_transpose[0][0]',    
                                                                     'conv2d_271[0][0]']          
                                                                                                  
 activation_51 (Activation)  (None, 4, 4, 512)            0         ['add[0][0]']                 
          

                                                                                                  
 up_sampling2d_59 (UpSampli  (None, 16, 16, 512)          0         ['batch_normalization_273[0][0
 ng2D)                                                              ]']                           
                                                                                                  
 batch_normalization_275 (B  (None, 16, 16, 256)          1024      ['conv2d_281[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 concatenate_21 (Concatenat  (None, 16, 16, 768)          0         ['up_sampling2d_59[0][0]',    
 e)                                                                  'batch_normalization_275[0][0
                                                                    ]']                           
          

 batch_normalization_282 (B  (None, 32, 32, 64)           256       ['conv2d_291[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_59 (Activation)  (None, 32, 32, 64)           0         ['batch_normalization_282[0][0
                                                                    ]']                           
                                                                                                  
 conv2d_293 (Conv2D)         (None, 32, 32, 64)           4160      ['activation_59[0][0]']       
                                                                                                  
 conv2d_transpose_3 (Conv2D  (None, 32, 32, 64)           36928     ['conv2d_293[0][0]']          
 Transpose)                                                                                       
          

In [10]:
def Attention_ResUNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):

    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    # input data
    # dimension of the image depth
    inputs = layers.Input(input_shape, dtype=tf.float32)
    axis = 3

    # Downsampling layers
    # DownRes 1, double residual convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=axis)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=axis)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=axis)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=axis)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=axis)(conv_final)
    conv_final = layers.Activation('sigmoid')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="AttentionResUNet")
    return model

In [93]:
model.compile(optimizer=Adam(lr = 1e-2), loss=BinaryFocalLoss(gamma=2), 
              metrics=['accuracy'])

In [94]:
unet_history = model.fit(X1, y1, 
                    verbose=1,
                    batch_size = 8,
                    validation_data=(X_test1, y_test ), 
                    shuffle=False,
                    epochs=50)

Epoch 1/50
102/102 [==============================] - 235s 2s/step - loss: 0.1848 - accuracy: 0.3005 - val_loss: 0.4335 - val_accuracy: 0.2579
Epoch 2/50
102/102 [==============================] - 212s 2s/step - loss: 0.1634 - accuracy: 0.2979 - val_loss: 0.2653 - val_accuracy: 0.2676
Epoch 3/50
102/102 [==============================] - 208s 2s/step - loss: 0.1534 - accuracy: 0.2832 - val_loss: 0.1809 - val_accuracy: 0.2673
Epoch 4/50
102/102 [==============================] - 229s 2s/step - loss: 0.1479 - accuracy: 0.2720 - val_loss: 0.1664 - val_accuracy: 0.2787
Epoch 5/50
102/102 [==============================] - 242s 2s/step - loss: 0.1436 - accuracy: 0.2684 - val_loss: 0.1659 - val_accuracy: 0.2796
Epoch 6/50
102/102 [==============================] - 241s 2s/step - loss: 0.1415 - accuracy: 0.2669 - val_loss: 0.2340 - val_accuracy: 0.2558
Epoch 7/50
102/102 [==============================] - 234s 2s/step - loss: 0.1438 - accuracy: 0.2644 - val_loss: 0.1383 - val_accuracy: 0.2963